# 06 - Weather time series modeling
ARIMA / SARIMA / SARIMAX + ML regressors (lags, rolling features) on 3h temperature series.

In [1]:
from pathlib import Path
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

PROJECT_ROOT = Path('..').resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from Model.time_series import (
    FeatureBuilder,
    OpenMeteoHistoricalRepository,
    TimeSeriesPreprocessor,
    WeatherQuery,
    ARIMAModel,
    SARIMAModel,
    SARIMAXModel,
    RegressionModel,
    GradientBoostingModel,
    compute_metrics,
    train_test_split_time,
)


## Load data (from CSV or fetch fresh)
Prefers the processed 3h dataset saved by notebook 05.

In [2]:
CITY = 'Ajaccio'
LATITUDE = 41.5536
LONGITUDE = 8.4413
START_DATE = '2014-01-01'
END_DATE = '2024-12-31'
DATA_PATH = PROJECT_ROOT / 'Data' / 'processed' / f'weather_{CITY.lower()}_3h.csv'

if DATA_PATH.exists():
    df = pd.read_csv(DATA_PATH)
else:
    repo = OpenMeteoHistoricalRepository(default_hourly=['temperature_2m'])
    query = WeatherQuery(latitude=LATITUDE, longitude=LONGITUDE, start_date=START_DATE, end_date=END_DATE)
    raw_df, _ = repo.fetch(query)
    preproc = TimeSeriesPreprocessor(step_hours=3)
    df = preproc.run(raw_df)
df['date'] = pd.to_datetime(df['date'], utc=True)
df.head()


,temperature_2m,date
0,5.563334,2014-01-01 00:00:00+00:00
1,4.863334,2014-01-01 03:00:00+00:00
2,4.980000,2014-01-01 06:00:00+00:00
3,7.413333,2014-01-01 09:00:00+00:00
4,9.213333,2014-01-01 12:00:00+00:00


## Feature engineering (lags, rolling stats, time encodings)
Builds supervised table for regression models.

In [3]:
feat_builder = FeatureBuilder(target_col='temperature_2m', time_col='date')
feature_df = feat_builder.build(df)
feature_df.head()


,temperature_2m,date,hour,dayofweek,month,sin_day,cos_day,temperature_2m_lag_1,temperature_2m_lag_2,temperature_2m_lag_3,temperature_2m_lag_4,temperature_2m_lag_8,temperature_2m_roll_mean_2,temperature_2m_roll_mean_4,temperature_2m_roll_mean_8
0,10.130000,2014-01-02 00:00:00+00:00,0,3,1,0.000000e+00,1.000000e+00,9.796666,9.863333,10.230000,9.213333,5.563334,9.963333,10.005000,8.311250
1,9.546667,2014-01-02 03:00:00+00:00,3,3,1,7.071068e-01,7.071068e-01,10.130000,9.796666,9.863333,10.230000,4.863334,9.838334,9.834167,8.896667
2,8.763334,2014-01-02 06:00:00+00:00,6,3,1,1.000000e+00,6.123234e-17,9.546667,10.130000,9.796666,9.863333,4.980000,9.155000,9.559167,9.369583
3,8.863333,2014-01-02 09:00:00+00:00,9,3,1,7.071068e-01,-7.071068e-01,8.763334,9.546667,10.130000,9.796666,7.413333,8.813333,9.325833,9.550833
4,10.280000,2014-01-02 12:00:00+00:00,12,3,1,1.224647e-16,-1.000000e+00,8.863333,8.763334,9.546667,10.130000,9.213333,9.571667,9.363333,9.684167


## Train / test split (time-based)
Keep the last week (8 samples/day * 7 days = 56) as holdout.

In [4]:
train_df, test_df = train_test_split_time(feature_df, test_size=56)
y_train = train_df['temperature_2m']
y_test = test_df['temperature_2m']


## ARIMA / SARIMA
Baseline ARIMA (p,d,q) and seasonal SARIMA (P,D,Q,s) on the aggregated series.

In [5]:
arima = ARIMAModel(order=(3, 0, 2))
arima.fit(train_df)
arima_pred = arima.predict(test_df)
arima_metrics = compute_metrics(y_test, arima_pred)
arima_metrics


c:\Users\lmanuelli\Projet\manuelli_laurent_projet_final\.venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\lmanuelli\Projet\manuelli_laurent_projet_final\.venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\lmanuelli\Projet\manuelli_laurent_projet_final\.venv\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


{'rmse': 5.954139618818157,
 'mae': 5.381312865351451,
 'mape': 439.76911037639684}

In [6]:
sarima = SARIMAModel(order=(2, 0, 2), seasonal_order=(1, 1, 1, 8))
sarima.fit(train_df)
sarima_pred = sarima.predict(test_df)
sarima_metrics = compute_metrics(y_test, sarima_pred)
sarima_metrics


{'rmse': 6.142646931644555,
 'mae': 5.644948745901898,
 'mape': 407.44854722690735}

## SARIMAX with exogenous
Use time features (sin/cos, month, etc.) as exogenous variables.

In [7]:
exog_cols = [c for c in feature_df.columns if c not in ['temperature_2m']]
sarimax = SARIMAXModel(order=(1, 0, 1), seasonal_order=(1, 1, 1, 8))
sarimax.fit(train_df[['temperature_2m']], exog=train_df[exog_cols])
sarimax_pred = sarimax.predict(test_df[['temperature_2m']], exog=test_df[exog_cols])
sarimax_metrics = compute_metrics(y_test, sarimax_pred)
sarimax_metrics


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

## ML regressors (lags + time features)
Compare RandomForest and GradientBoosting baselines.

In [8]:
feature_cols = [c for c in feature_df.columns if c != 'temperature_2m']
rf = RegressionModel(feature_cols=feature_cols)
rf.fit(train_df)
rf_pred = rf.predict(test_df)
rf_metrics = compute_metrics(y_test, rf_pred)
rf_metrics


TypeError: float() argument must be a string or a real number, not 'Timestamp'

In [ ]:
gbr = GradientBoostingModel(feature_cols=feature_cols)
gbr.fit(train_df)
gbr_pred = gbr.predict(test_df)
gbr_metrics = compute_metrics(y_test, gbr_pred)
gbr_metrics


## Residual analysis
Inspect error distribution for the best model (replace here with SARIMA).

In [ ]:
residuals = y_test - sarima_pred
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
residuals.plot(ax=axes[0], title='Residuals over time')
axes[1].hist(residuals, bins=20)
axes[1].set_title('Residual distribution')
plt.tight_layout()
